## Task two

### Stage three

Export a the parquet files to a DuckDB (embedded DB) database, in order to serve the data via an API.  

SQLite might have been an alternative here, but exposing the service the parquet files via FastAPI or Flask might was problematic, especially as a table join was needed.

In [2]:
#from pyspark.context import SparkContext
#from pyspark.conf import SparkConf
#from pyspark.sql import functions as f
#
#conf = SparkConf()\
#    .setAppName('imdb-munging')\
#    .setMaster('local[*]')\
#    .setExecutorEnv("spark.driver.memory", "15g")
#
#sc = SparkContext.getOrCreate(conf=conf)


In [48]:
#from duckdb.experimental.spark.sql import SparkSession as session
#import duckdb.experimental.spark.sql.functions as f
#
#spark = session.builder.getOrCreate()

In [1]:
import duckdb

In [ ]:
db_file = "../duckdb/films.duckdb"

ddb_conf = {'access_mode': 'read_only',}
# to use a database file (not shared between processes)
con = duckdb.connect(db_file, config = ddb_conf)

In [23]:
#con.sql("SELECT current_setting('threads') AS threads;")
con.sql("SELECT * FROM duckdb_settings()")

┌─────────────────────────────────────────────┬───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬─────────┐
│                    name                     │   value   │                                                  description                                                   │ input_type │  scope  │
│                   varchar                   │  varchar  │                                                    varchar                                                     │  varchar   │ varchar │
├─────────────────────────────────────────────┼───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────┼─────────┤
│ access_mode                                 │ read_only │ Access mode of the database (AUTOMATIC, READ_ONLY or READ_WRITE)                                               │ VARCHAR    │ GLOBAL  │
│ allocator_backgrou

In [4]:
# load the IMDb films data prepared in previous task
#files = "../output/%s/*.parquet" % "films"
#df_film = duckdb.read_parquet(files)
# SELECT * FROM read_parquet('genres/*/*.parquet', hive_partitioning = true);
#duckdb.sql("SELECT * FROM '..output/films/*.parquet'")

In [ ]:
# Ingest all the film details data prepared in spark
parquet_files = "../output/%s/*.parquet" % "films"

# Change `read_only` to False
with duckdb.connect(db) as con:
    con.sql(f'CREATE TABLE films AS SELECT * FROM read_parquet("{parquet_files}")')
    con.sql("DESCRIBE films").show()


In [ ]:
# ingest the cosine similarity data prepared in spark
parquet_files = "../output/%s/*.parquet" % "csfilm"
ingest_query = f'SELECT * FROM read_parquet("{parquet_files}")'

with duckdb.connect(db) as con:
    con.sql(f'CREATE OR REPLACE TABLE films_cos_sim AS {ingest_query}')
    con.sql("DESCRIBE films_cos_sim").show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [25]:
#with duckdb.connect(db_file) as con:
columns = con.execute("SELECT DISTINCT(name) FROM pragma_table_info('films')").fetchall()
con.sql("DESCRIBE films").show()
con.sql("DESCRIBE films_cos_sim").show()
#con.table("films_cos_sim").show()


┌─────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type  │  null   │   key   │ default │  extra  │
│   varchar   │   varchar    │ varchar │ varchar │ varchar │ varchar │
├─────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ film_id     │ INTEGER      │ YES     │ NULL    │ NULL    │ NULL    │
│ title       │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ year        │ DATE         │ YES     │ NULL    │ NULL    │ NULL    │
│ duration    │ INTEGER      │ YES     │ NULL    │ NULL    │ NULL    │
│ genres      │ VARCHAR[]    │ YES     │ NULL    │ NULL    │ NULL    │
│ rating      │ DECIMAL(4,2) │ YES     │ NULL    │ NULL    │ NULL    │
│ vote_count  │ INTEGER      │ YES     │ NULL    │ NULL    │ NULL    │
│ persons     │ VARCHAR[]    │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴──────────────┴─────────┴─────────┴─────────┴─────────┘

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ colu

In [44]:
# df_film.describe()

┌─────────┬────────────────────┬────────────────────┬────────────┬────────────────────┬───────────┬────────────────────┬───────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  aggr   │      film_id       │       title        │    year    │      duration      │  genres   │       rating       │    vote_count     │                                                                                                                                    persons                                                                                                                                    │
│ varchar │       double       │      varchar       │  varchar   │       double       │  varchar  │       double       │      double       │                                  

In [74]:
# A few sample queries

#df_film.filter('genres = "Documentary"').limit(3).show()

#with duckdb.connect(db) as con:

#con.sql("SELECT * FROM films WHERE year ='2024-01-01'").show()
#con.sql("SELECT * FROM films WHERE list_has_any(genres, ['Romance', 'Documentary']) ORDER BY rating DESC").limit(20).show()
#con.sql("SELECT * FROM films WHERE list_has_any(persons, ['Suzanne Collins']) ORDER BY rating DESC").limit(20).show()
#con.sql("SELECT * FROM films WHERE list_contains(persons, 'Suzanne Collins') ORDER BY rating DESC").limit(20).show()    
#con.sql("SELECT * FROM films WHERE list_contains(persons, 'Ridley Scott') ORDER BY title").show(max_rows=50)  
#con.sql("SELECT genres, count() AS count FROM films GROUP BY genres ORDER BY count DESC").show(max_rows=50)
#con.sql("SELECT * FROM films WHERE title LIKE '%Alien%' ORDER BY title").show(max_rows=50)  
#con.sql("SELECT * FROM films WHERE regexp_matches(title, '.*Alien.*') ORDER BY title").show(max_rows=50)
con.sql("SELECT * FROM films WHERE regexp_matches(title, '.*Home.Alone.*') ORDER BY title").show(max_rows=50)  

#con.sql("SELECT * FROM films WHERE regexp_matches(persons, '.*Culkin') ORDER BY title").show(max_rows=50)  

#df_film.count() # 39_427



┌─────────┬──────────────────────────────────┬────────────┬──────────┬────────────────────────────┬──────────────┬────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ film_id │              title               │    year    │ duration │           genres           │    rating    │ vote_count │                                                                                                                                        persons                                                                                                                                         │
│  int32  │             varchar              │    date    │  int32   │         varchar[]          │ decimal(4,2) │   int32    │                                                       

## Function to detect similarities between films

In [67]:
from pandas import DataFrame

def similar_films(film_id:int, threshold:float=0.1) -> DataFrame:
    #with duckdb.connect(db_file, read_only=True) as con:
        rdf = con.sql("""    
            SELECT similarity, films.film_id, title, year, duration, genres, rating, vote_count, persons
              FROM films_cos_sim AS sim
         LEFT JOIN films ON sim.other_id == films.film_id
             WHERE sim.film_id == %s
               AND sim.similarity >= %s
          ORDER BY sim.similarity DESC
             LIMIT 10
          """ % (film_id, threshold) ).df()

        return rdf

def get_film(film_id:int) -> DataFrame:
    rdf = con.sql("""
        SELECT *
          FROM films
         WHERE film_id == %s
      """ % (film_id) ).df()

    return rdf

In [68]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
display( get_film(99785) )

#similar_films(88763) # "BTTF"
#similar_films(990372, 0.45) # "Detective Conan"
#similar_films(462699) # "Conan the Future Boy"
#similar_films(1392170) # "The Hunger Games"
#similar_films(78748) # "Alien"
#similar_films(90605) # "Aliens"
similar_films(99785) # Home Alone
#similar_films(11286314) # "Don't Look Up"
#similar_films(15410318, 0.1) # "Amy's bucket list"
#similar_films(1517268, 0.05) # "Barbie"
#similar_films(120915, 0.1) # Star Wars I
#similar_films(133093, 0.2) # The Matrix
#similar_films(6791350) # Guardians of the Galaxy 3
#similar_films(film_id=11286314, threshold=0.1)
#similar_films(120915, 0.97) # Star War

,film_id,title,year,duration,genres,rating,vote_count,persons
0,99785,Home Alone,1990-01-01,103,"[Comedy, Family]",7.7,675425,"[John Heard, Raja Gosnell, Daniel Stern, John Hughes, Devin Ratray, Hillary Wolf, Roberts Blossom, Chris Columbus, John Muto, John Williams, Gerry Bamman, Jane Jenkins, Janet Hirshenson, Macaulay Culkin, Julio Macat, Angela Goethals, Joe Pesci, Catherine O'Hara]"


,similarity,film_id,title,year,duration,genres,rating,vote_count,persons
0,0.717906,104431,Home Alone 2: Lost in New York,1992-01-01,120,"[Adventure, Comedy, Crime]",6.9,416350,"[John Heard, Raja Gosnell, Daniel Stern, John Hughes, Devin Ratray, Hillary Wolf, Chris Columbus, John Williams, Gerry Bamman, Jane Jenkins, Janet Hirshenson, Sandy Veneziano, Macaulay Culkin, Julio Macat, Michael C. Maronna, Joe Pesci, Maureen Elisabeth Shay, Catherine O'Hara]"
1,0.324695,102598,Only the Lonely,1991-01-01,104,"[Comedy, Romance]",6.4,12669,"[Raja Gosnell, Maurice Jarre, Maureen O'Hara, Hunt Lowry, John Hughes, Chris Columbus, Kevin Dunn, Jim Belushi, John Muto, Marvin J. McIntyre, Jane Jenkins, Ally Sheedy, Janet Hirshenson, John Candy, Julio Macat, Joe Greco, Milo O'Shea, Bert Remsen, Peter Teschner, Anthony Quinn]"
2,0.214061,110527,Miracle on 34th Street,1994-01-01,114,"[Drama, Family, Fantasy]",6.6,45628,"[Raja Gosnell, George Seaton, Doug Kraner, John Hughes, William Windom, Bruce Broughton, Jane Leeves, Richard Attenborough, Dylan McDermott, Elizabeth Perkins, Jane Jenkins, James Remar, Janet Hirshenson, Mara Wilson, Robert Prosky, J.T. Walsh, Julio Macat, Simon Jones, Valentine Davies, Les Mayfield]"
3,0.151786,107614,Mrs. Doubtfire,1993-01-01,125,"[Comedy, Drama]",7.1,299517,"[Angelo P. Graham, Raja Gosnell, Mark Radcliffe, Anne Haney, Harvey Fierstein, Leslie Dixon, Lisa Jakub, Matthew Lawrence, Donald McAlpine, Chris Columbus, Polly Holliday, Jane Jenkins, Anne Fine, Randi Mayem Singer, Janet Hirshenson, Mara Wilson, Robert Prosky, Pierce Brosnan, Sally Field, Robin Williams, Howard Shore, Marsha Garces Williams]"
4,0.150241,241527,Harry Potter and the Sorcerer's Stone,2001-01-01,152,"[Adventure, Family, Fantasy]",7.7,884416,"[Richard Francis-Bruce, Rupert Grint, Fiona Shaw, Richard Griffiths, Daniel Radcliffe, Maggie Smith, Karen Lindsay-Stewart, Chris Columbus, Stuart Craig, David Heyman, Robbie Coltrane, Saunders Triplets, Susie Figgis, John Williams, Jane Jenkins, J.K. Rowling, Harry Melling, Janet Hirshenson, John Seale, Richard Harris, Emma Watson, Steve Kloves]"
5,0.112831,107985,Rookie of the Year,1993-01-01,103,"[Comedy, Family, Fantasy]",6.1,31793,"[Raja Gosnell, Albert Hall, Robert Harper, Donn Cambern, Daniel Stern, Gary Busey, Linda Lowy, Jack N. Green, Steven J. Jordan, Robert Hy Gorman, Patrick LaBrecque, Eddie Bracken, Dan Hedaya, Thomas Ian Nicholas, Bruce Altman, Amy Morton, Bill Conti, Sam Harper]"
6,0.108662,110771,The Paper,1994-01-01,112,"[Comedy, Drama]",6.7,16388,"[Frederick Zollo, Randy Newman, Todd Hallowell, Stephen Koepp, Ron Howard, Glenn Close, Spalding Gray, Daniel P. Hanley, Mike Hill, Randy Quaid, Jason Alexander, Marisa Tomei, Jane Jenkins, Janet Hirshenson, Jason Robards, Robert Duvall, John Seale, Michael Keaton, Lynne Thigpen, David Koepp, Brian Grazer, Catherine O'Hara]"
7,0.106963,109190,Baby's Day Out,1994-01-01,99,"[Adventure, Comedy, Crime]",6.2,57174,"[Matthew Glave, John Neville, Joe Pantoliano, Joe Mantegna, Patrick Read Johnson, Doug Kraner, John Hughes, Bruce Broughton, Adam Robert Worton, Cynthia Nixon, Lara Flynn Boyle, Jane Jenkins, Richard Vane, Jacob Joseph Worton, Janet Hirshenson, Thomas E. Ackerman, David Rawlins, Fred Thompson, Brian Haley]"
8,0.103805,986263,Surrogates,2009-01-01,89,"[Action, Sci-Fi, Thriller]",6.3,181716,"[Rosamund Pike, Kevin Stitt, James Francis Ginty, Jack Noseworthy, Ving Rhames, Michael Cudlitz, Boris Kodjoe, Jeff Mann, Bruce Willis, John Brancato, Devin Ratray, James Cromwell, Todd Lieberman, Oliver Wood, Jonathan Mostow, Michelle Lewitt, Brett Weldele, Max Handelman, Robert Venditti, David Hoberman, Jane Jenkins, Janet Hirshenson, Radha Mitchell, Michael Ferris, Richard Marvin]"
9,0.102584,101786,Dutch,1991-01-01,107,"[Comedy, Drama]",6.5,14768,"[Ed O'Neill, Peter Faiman, L. Scott Caldwell, Kathleen Freeman, Christopher McDonald, Ethan Embry, Ari Meyers, Elizabeth Daily, Cedering Fox, John Hughes, Lisa Figus, Paul Hirsch, Stan Jolley, Alan Silve

In [84]:
#sc.stop()